In [ ]:
import json
import os
# read json file
with open('../Data/stations.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

for station_name, station_info in data.items():
    print("Station name:", station_name)
    print("district:", station_info['district'])
    print("introduce:", station_info['intro'])

    # print bus info
    print("bus info:")
    for bus_info in station_info['bus_info']:
        print("- chukou:", bus_info['chukou'])
        for bus_out_info in bus_info['busOutInfo']:
            print("  - busInfo:", bus_out_info['busInfo'])
            print("  - busName:", bus_out_info['busName'])

    # print outt info
    print("outt info:")
    for out_info in station_info['out_info']:
        print("- outt:", out_info['outt'])
        print("- textt:", out_info['textt'])

    print("\n")


# read JSON file
with open('../Data/cards.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

for record in data:
    print("code:", record['code'])
    print("money:", record['money'])
    print("create_time:", record['create_time'])



import psycopg
import os
db_name = os.environ['DB_NAME']
db_user = os.environ['DB_USER']
db_password = os.environ['DB_PASSWORD']
db_host = os.environ['DB_HOST']
db_port = os.environ['DB_PORT']
print(db_name, db_user, db_password, db_host, db_port)
conn = psycopg.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
class DatabaseManipulation:
    def __init__(self):
        self.conn = conn
        self.cur = self.conn.cursor()
    def findMovieById(self, id):
        query = "SELECT m.title, \
                c.country_name, \
                m.year_released, \
                m.runtime FROM movies m JOIN countries c ON m.country = c.country_code \
                WHERE m.movieid = %s"
        self.cur.execute(query, (id,))
        result = self.cur.fetchone()
        return result
    
q = DatabaseManipulation()
q.cur = conn.cursor()
print(q.findMovieById(2))


In [14]:
import json
import psycopg
import os
import subprocess
import re

def extract_env_vars():
    with open('../setenv.sh', 'r') as file:
        lines = file.readlines()
    env_vars = {}
    for line in lines:
        match = re.match(r"export (\w+)='?([^']*)'?", line)
        if match:
            env_vars[match.group(1)] = match.group(2).strip()
    print(env_vars)
    return env_vars

# Function to connect to the PostgreSQL database
def create_conn():
    env_vars = extract_env_vars()
    db_name = env_vars['DB_NAME']
    db_user = env_vars['DB_USER']
    db_password = env_vars['DB_PASSWORD']
    db_host = env_vars['DB_HOST']
    db_port = env_vars['DB_PORT']
    conn = psycopg.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
    return conn

# Function to create tables in the PostgreSQL database
def create_tables(conn):
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS stations (
            station_name TEXT PRIMARY KEY,
            district TEXT,
            intro TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS bus_info (
            station_name TEXT,
            chukou TEXT,
            bus_name TEXT,
            bus_info TEXT,
            FOREIGN KEY (station_name) REFERENCES stations (station_name)
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS out_info (
            station_name TEXT,
            outt TEXT,
            textt TEXT,
            FOREIGN KEY (station_name) REFERENCES stations (station_name)
        )
    """)
    conn.commit()

# Function to insert data into the tables
def insert_data(conn, data):
    cur = conn.cursor()
    for station_name, station_info in data.items():
        cur.execute("""
            INSERT INTO stations (station_name, district, intro) 
            VALUES (%s, %s, %s)
        """, (station_name, station_info['district'], station_info['intro']))
        for bus_info in station_info['bus_info']:
            for bus_out_info in bus_info['busOutInfo']:
                cur.execute("""
                    INSERT INTO bus_info (station_name, chukou, bus_name, bus_info) 
                    VALUES (%s, %s, %s, %s)
                """, (station_name, bus_info['chukou'], bus_out_info['busName'], bus_out_info['busInfo']))
        for out_info in station_info['out_info']:
            cur.execute("""
                INSERT INTO out_info (station_name, outt, textt) 
                VALUES (%s, %s, %s)
            """, (station_name, out_info['outt'], out_info['textt']))
    conn.commit()

# Read the JSON file
with open('../Data/stations.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Call the function to create tables
conn = create_conn()
create_tables(conn)

# Call the function to insert data
insert_data(conn, data)

{'DB_NAME': 'cscs307', 'DB_USER': 'superlucky', 'DB_PASSWORD': '', 'DB_HOST': 'localhost', 'DB_PORT': '5432'}
